# Elastic Weight Consolidation

In [2]:
%load_ext autoreload
%autoreload 2

# Global imports

In [19]:
import sys

from torch.optim import SGD
from torch import tensor

# Local imports

In [3]:
sys.path.append('..')

from base_code.models.poc import NeuralNetwork as NN
from base_code.datasets import DryBeansDataset as DBD
from base_code.train_test import train, test

from base_code.preprocessing.nomalize_standarizer import pipeline as normalize_standarize
from base_code.dataloaders.base import ContinualLearningDataLoader as DataLoader
from base_code.losses.ewc import EWC

# Data preprocessing

In [4]:
dataset = DBD()

In [5]:
dataset.get_task_range(0, 2).labels

array([[1., 0., 0., ..., 0., 0., 0.],
       [1., 0., 0., ..., 0., 0., 0.],
       [1., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 1., 0., ..., 0., 0., 0.],
       [0., 1., 0., ..., 0., 0., 0.],
       [0., 1., 0., ..., 0., 0., 0.]])

In [6]:
pipeline = normalize_standarize(dataset.features)

In [7]:
X = pipeline.transform(dataset.features)
dataset.features = X

# DataLoader settings

In [8]:
train_dataloader = DataLoader(dataset, batch_size=10, shuffle=True)

# Neural Netwrok

In [9]:
model = NN(features_shape=dataset.features_shape, output_shape=dataset.output_shape)

In [22]:
loss_fn = EWC(importance=1)
optimizer = SGD(model.parameters(), lr=0.001, momentum=0.9)

# Train - Test

## Concurrent

In [11]:
epochs = 5
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train(model, train_dataloader, loss_fn, optimizer)
    test(model, train_dataloader, loss_fn)
print("Done!")

Epoch 1
-------------------------------
loss: 1.938606  [   10/13611]
loss: 1.926388  [ 1010/13611]
loss: 1.933463  [ 2010/13611]
loss: 1.926143  [ 3010/13611]
loss: 1.926182  [ 4010/13611]
loss: 1.917880  [ 5010/13611]
loss: 1.893715  [ 6010/13611]
loss: 1.652866  [ 7010/13611]
loss: 1.750546  [ 8010/13611]
loss: 1.654315  [ 9010/13611]
loss: 1.549801  [10010/13611]
loss: 1.812034  [11010/13611]
loss: 1.818748  [12010/13611]
loss: 1.748461  [13010/13611]
Test Error: 
 Accuracy: 38.1%, Avg loss: 1.761336 

Epoch 2
-------------------------------
loss: 1.897772  [   10/13611]
loss: 1.733104  [ 1010/13611]
loss: 1.798549  [ 2010/13611]
loss: 1.718744  [ 3010/13611]
loss: 1.582367  [ 4010/13611]
loss: 1.888248  [ 5010/13611]
loss: 1.880362  [ 6010/13611]
loss: 1.779645  [ 7010/13611]
loss: 1.794405  [ 8010/13611]
loss: 1.629606  [ 9010/13611]
loss: 1.616499  [10010/13611]
loss: 1.873621  [11010/13611]
loss: 1.739490  [12010/13611]
loss: 1.895439  [13010/13611]
Test Error: 
 Accuracy: 38.8

## Sequential

In [12]:
epochs = 5
for task_id in range(train_dataloader.get_tasks_length()):
    print(f"Task {task_id+1}\n-------------------------------")
    for t in range(epochs):
        print(f"Epoch {t+1}\n-------------------------------")
        train(model, train_dataloader.get_task(task_id), loss_fn, optimizer)
        test(model, train_dataloader.get_task_range(0, task_id+1), loss_fn)

Task 1
-------------------------------
Epoch 1
-------------------------------
loss: 1.276212  [   10/ 2027]
loss: 1.297114  [ 1010/ 2027]
loss: 1.187896  [ 2010/ 2027]
Test Error: 
 Accuracy: 98.7%, Avg loss: 1.201651 

Epoch 2
-------------------------------
loss: 1.204704  [   10/ 2027]
loss: 1.175839  [ 1010/ 2027]
loss: 1.219845  [ 2010/ 2027]
Test Error: 
 Accuracy: 99.4%, Avg loss: 1.185685 

Epoch 3
-------------------------------
loss: 1.166602  [   10/ 2027]
loss: 1.170341  [ 1010/ 2027]
loss: 1.174511  [ 2010/ 2027]
Test Error: 
 Accuracy: 99.8%, Avg loss: 1.178281 

Epoch 4
-------------------------------
loss: 1.179792  [   10/ 2027]
loss: 1.168353  [ 1010/ 2027]
loss: 1.172774  [ 2010/ 2027]
Test Error: 
 Accuracy: 99.9%, Avg loss: 1.174184 

Epoch 5
-------------------------------
loss: 1.165649  [   10/ 2027]
loss: 1.165880  [ 1010/ 2027]
loss: 1.173934  [ 2010/ 2027]
Test Error: 
 Accuracy: 100.0%, Avg loss: 1.171874 

Task 2
-------------------------------
Epoch 1
---